<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/0321_HW_Unsupervised_Learning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **가우시안 혼합 (Gaussian Mixture)**

샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델

하나의 가우시안 분포에서 생성된 모든 샘플은 하나의 클러스터를 형성하며, 일반적으로 클러스터는 타원형임

In [1]:
# 데이터 로드
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise= 0.05)

In [2]:
# GMM 구현
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [3]:
# 파라미터 확인
print('가중치: {0}, \n평균: {1}, \n공분산 행렬: {2}'.
      format(gm.weights_, gm.means_, gm.covariances_))

가중치: [0.60447211 0.18656204 0.20896585], 
평균: [[ 0.53032487  0.22306046]
 [ 1.78245549 -0.02871566]
 [-0.72964534  0.57189989]], 
공분산 행렬: [[[ 0.17724481 -0.11004126]
  [-0.11004126  0.29289199]]

 [[ 0.04010086  0.05129794]
  [ 0.05129794  0.08163791]]

 [[ 0.06014152  0.06769829]
  [ 0.06769829  0.0929207 ]]]


## **기댓값-최대화 알고리즘**

 GMM이 파라미터를 찾는 방법, 클러스터 파라미터를 랜덤하게 초기화하고 수렴할 때까지 반복한다는 점에서 K-평균과 유사


Step 0) 클러스터 파라미터를 랜덤하게 초기화

Step 1) 샘플을 클러스터에 할당함 (기댓값 단계)

Step 2) 클러스터를 업데이트 (최대화 단계)

=> k-평균의 일반화로 간주할 수 있음

*k-평균 알고리즘과의 차이*

- EM (Expectation-Maximization) 알고리즘은 하드 클러스터 할당이 아니라 소프트 글러스터 할당을 사용함

- 즉, 기댓값 단계에서 알고리즘은 현재 클러스터 파라미터에 기반하여 각 클러스터에 속할 **확률**을 예측

- 그다음 최대화 단계에서 각 클러스터가 데이터셋에 있는 모든 샘플을 사용해 업데이트 됨
    - 이때 클러스터에 속할 추정 확률로 샘플에 가중치가 적용됨
    - 이 확률을 샘플에 대한 클러스터의 **책임** 이라고 부르며, 업데이트는 책임이 가장 많은 샘플에 크게 영향을 받음

이또한 k-평균처럼 나쁜 솔루션으로 수렴할 수 있으므로 여러 번 실행하여 가장 좋은 솔루션을 선택해야함

n_init 옵션을 통해 여러 번 실행하게 할 수 있음 (n_init의 기본값이 1이므로 주의!!)

In [4]:
# 알고리즘의 수렴 여부와 반복 횟수 확인
print(gm.converged_)
print(gm.n_iter_)

True
17


In [5]:
# 새로운 샘플에 대한 예측
# 클러스터에 할당 (하드 군집)
gm.predict(X)[:5]

array([0, 1, 2, 0, 0])

In [6]:
# 특정 클러스터에 속할 확률 예측 (소프트 군집)
gm.predict_proba(X)

array([[1.00000000e+000, 9.17713232e-043, 2.10146246e-055],
       [3.40086985e-003, 9.96599130e-001, 2.45269565e-192],
       [4.47053878e-002, 1.04664870e-227, 9.55294612e-001],
       ...,
       [3.26074922e-001, 1.23664243e-200, 6.73925078e-001],
       [1.01486536e-003, 9.98985135e-001, 1.05592496e-165],
       [1.00000000e+000, 1.80733611e-044, 1.54835266e-053]])

가우시안 혼합 모델은 **생성 모델 (generative model)**임. 즉, 이 모델에서 새로운 샘플을 만들 수 있다!

In [7]:
X_new, y_new = gm.sample(6)

# 반환된 샘플은 클러스터 인덱스 순으로 정렬되어 있음
print(X_new)
print()
print(y_new)

[[ 0.91230523 -0.42480364]
 [ 0.19020576  0.33301912]
 [ 0.2281136  -0.35476291]
 [-0.03742326  0.97767243]
 [ 0.88140111 -0.77261074]
 [ 0.17726254  0.47119739]]

[0 0 0 0 0 0]


In [8]:
# score_smaples() 메서드 : 샘플의 위치에서의 log(pdf) 값을 예측
# 해당 점수의 지숫값을 계산하면 샘플 위치에서의 pdf 값을 얻을 수 있음
print(gm.score_samples(X).shape)
gm.score_samples(X)[:5]

(1000,)


array([-1.65275662, -0.95799496, -0.52208317, -1.67846625, -1.73845474])

훈련된 가우시안 혼합 모델의 클러스터 평균, 결정 경계, 밀도 등고선
![GMM](https://blog.kakaocdn.net/dn/FOS7u/btqVFBq78Hf/WkuTOGkeUNlXhY4ps1ktTK/img.png)

covariance_type 파라미터

default : "full" 이며 각 클러스터의 모양/크기/방향에 제약 X

**[제약]**
- spherical : 모든 클러스터가 원형 (지름, 즉 분산은 다를 수 있음)

- diag : 클러스터가 크기에 상관없이 어떠한 타원형도 가능, but 타원의 축은 좌표 축과 나란해야 함 (즉, 공분산 행렬이 대각행렬)

- tied : 모든 클러스터가 동일한 타원 모양,크기,방향을 가짐 (모든 클러스터의 공분산 행렬 동일)

## **가우시안 혼합을 사용한 이상치 탐지**

밀도가 낮은 지역에 있는 모든 샘플을 이상치로 판단하는 원리 >> 임계값 지정이 필요함

In [9]:
# 밀도 임계값 4% 지정 후 이상치 탐지
import numpy as np

densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

![anomalies](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fo8lhx%2FbtraFFGccCi%2Fi9PXuzdez6PchGvS8t2eyk%2Fimg.png)

## **클러스터 개수 선택하기**

k-평균에서의 클러스터 개수 선택 방법

- 이너셔 그래프를 그려 그래프가 느리게 감소하기 시작하는 '엘보'를 클러스터의 개수로 선정
- 실루엣 점수 및 실루엣 다이어그램 활용

=> 클러스터가 타원형이나 크기가 다를 때 안정적이지 않으므로 GMM에서는 사용 불가

=> **AIC, BIC** 를 사용

**BIC (Bayseian information criterion) / AIC (Akaike information criterion)**

$BIC = log(m)p - 2log(\hat{L})$      

$AIC = 2p - 2log(\hat{L})$

- m : 샘플 수
- p : 모델이 학습할 파라미터 개수
- $\hat{L}$ : 모델의 likelihood function의 최댓값

AIC, BIC는 모두 학습할 파라미터가 많은 (=클러스터가 많은) 모델에 페널티를 가하고 데이터에 잘 학습하는 모델에 보상을 더함

AIC, BIC가 선택한 모델이 각각 다른 경우
- BIC의 모델이 AIC의 모델보다 간단한(=파라미터가 적은) 경향이 있음
- 따라서 데이터에 아주 잘 맞지는 않을 수 있음

In [10]:
# bic(), aic() 메서드로 계산 가능
print("BIC: {0:.4f}, AIC: {1:.4f}".format(gm.bic(X), gm.aic(X)))

BIC: 2787.9703, AIC: 2704.5385


![aic_bic](https://blog.kakaocdn.net/dn/ZztWM/btraLsTP1ca/9K7Sm2xCi1QV1Cy2uvAB20/img.png)

k=3이 최선

## **⭐Likelihood Function**

가능도 : 출력 x를 알고 있을 때 특정 파라미터 값 $\theta$가 얼마나 그럴듯한지 설명함

cf) 확률 : 미래 출력 x가 얼마나 그럴듯한지 설명 ($\theta$를 알고 있는 경우)

- 가능도 함수는 $\theta$에 대한 함수이며, *확률 분포가 아님*
- 데이터셋 X가 주어졌을 때 일반적으로 모델 파라미터에 대해 가장 그럴듯한 값을 예측
    - X에 대한 likelihood를 최대화하는 값(MLE)을 찾음
    - log likelihood를 maximize하는 값을 구하는 것이 더 쉬움

![likelihood](https://yunhwankim2.github.io/assets/images/likelihood_function_plot.png)

## **베이즈 가우시안 혼합 모델**

최적의 클러스터 개수를 수동으로 찾지 않고 불필요한 클러스터의 가중치를 0으로 (또는 0에 가깝게) 만드는 알고리즘

- n_components를 최적의 클러스터 개수보다 '크다'고 믿을 만한 값으로 지정
    - 자동으로 불필요한 클러스터를 제거하기 때문

In [11]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_, 2)

/usr/local/lib/python3.11/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


array([0.11, 0.16, 0.16, 0.11, 0.13, 0.14, 0.09, 0.  , 0.11, 0.  ])

3개의 클러스터는 필요 없음을 감지함

**가우시안 혼합 모델의 단점**

타원형 클러스터에 잘 작동함 > 다른 모양을 가진 데이터셋을 훈련하면 나쁜 결과를 얻음

![GMM2](https://blog.kakaocdn.net/dn/bWkeU2/btqVIb6KRJb/a1nhUKcKqpuEmlnSgjMFAK/img.png)

- GMM은 타원이 되도록 클러스터를 찾기 때문에 8개의 클러스터를 찾음
- 밀도 추정은 나쁘지 않으므로 이상치 감지를 위해서는 사용할 수 있을 것

## **이상치 탐지와 특이치 탐지를 위한 다른 알고리즘**

* PCA
* Fast-MCD
* 아이솔레이션 포레스트
* LOF (local outlier factor)
* one-class SVM